In [1]:
!pip install konlpy
import re
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 11.8MB/s 
     |████████████████████████████████| 3.8MB 57.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
reviews = pd.read_csv('drive/My Drive/datasets/review_coupang_1.csv')
reviews = reviews.loc[:,['content']]

In [3]:
reviews.sample(frac=1).reset_index(drop=True)

,content
0,양이 적다 맛은 괜찮은데.. 양이 너무 작아요. 2개 들어 있는데 다 먹어도 부족할...
1,신선하고 맛있어요 샐러리 통채로 사려하니 손질하기 귀찮기도 하고 양도 많을 것 같아...
2,간편식으로ㅠ부담없이 좋아요\n맛있어요..신선하고~~ 맛 만족도 예상보다 맛있어요
3,맛있어요 늘 맛이게 잘 먹고 있는 떡볶이입니다! 냉동실에 쟁여두고 뭐 먹을까 고민될...
4,고춧가루도 국산이고 새콤하고 와사비의 알싸한 맛까지 아주 좋아요.\n단짠 유행이지만...
...,...
146887,애기들이좋아할것같음 작아서 한입에먹기 편리하고 애기들이 좋아할것같아요
146888,너무 기대를 했는지 조금은 밍밍했어요 물보다 우유를 넣고 끓이면 좀 낫고 매운맛은 ...
146889,자취생 아이 어르신계신 집에서는 쟁여놓고 먹기 좋아요 간편하게 과일먹고싶을때 딱좋아
146890,오늘처럼 펑펑 눈오는 날..바로 시식해야겠어요 막내가 좋아하는 라면...저렴하게 잘...


In [4]:
!pip install chatspace

     |████████████████████████████████| 10.7MB 5.9MB/s 


In [7]:
# 띄어쓰기 보정 오픈소스
from chatspace import ChatSpace

spacer = ChatSpace()

Loading JIT Compiled ChatSpace Model


In [21]:
reviews = reviews.dropna()

In [ ]:
reviews

In [ ]:
test_data = []
for text in reviews['content']:
  test_data.append(spacer.space(text).replace('\n',''))

In [ ]:
test_data = pd.DataFrame(test_data, columns=['review'])
test_data

In [ ]:
keyword_list = {'맛': ['맛','불맛', '감칠맛', '쓴맛', '향', '단맛', '신맛', '냄새', '끝맛', '짠맛', '꼬순내', '햄맛',
                      '햄향', '닭맛', '불향', '풋내', '조미료', '끝향', '쏘스맛', '닭내', '파맛', '존맛', '핵존맛', '텁텁한맛'],
                '가격':['가격', '금액', '값', '단가', '할인', '가성', '득템', '창렬', '혜자', '정가', '사기'],
                '양':['용량', '양은', '양도', '양으로', '개수', '인분', '분량', '물량', '사이즈', '수량','남았', '남아'],
                '품질':['품질', '상태', '아이스', '얼음', '비닐', '아이스팩', '뾱뾱', '뽁뽁','드라이아이스', '아이스박스', '보냉', '상자',
                      '종이팩', '종이', '상온', '실온','상한',
                        '소재', '박스', '비닐봉투', '밀봉', '보냉팩', '배송', '발송', '택배', '배달', '검사', '퀄리티', '퀄러티','환경',
                       '검증', '신뢰', '신선',  '유통', '유효', '기한', '싱싱', '상한', '부패', '냄새'],
                '간편':['간편', '간단', '초보', '방법', '쉬움', '어려움', '단시간', '출퇴근시간', '간편식', '전문식','편리', '손쉽', '혼밥', '꽝손', 
                      '불편', '똥손', '금손', '편의', '손쉬'],
                '외관':['외관', '육안', '디자인', '실물', '겉보기', '보고', '보임', '볼때', '보고', '맛보다',
                       '푸디', '색감', '색깔', '모양새'],
                '웰빙':['웰빙', 'msg', '인스턴트', '나트륨', '특수', '식품', '즉석식품', '건강', '다이어트', '다이어터', '헬스', '첨가', '식단', '단백질',
                      '영양', '효과', '예방', '체중', '영양소', '탄수화물', '칼로리', '성분', '소화효소', '체중감량', '면역', '근육', '야맹증',
                      '질환', '보조', '미용', '운동', '에너지', '호르몬', '칼슘', '성인병', '갱년기', '성장', '비료', '노화', 
                      '치매', '효능', '염분', '두뇌', '불균형', '뇌', '알츠하이머', '코로나', '영양분', '소화','비만','뚱뚱', '살빼', '녹색','동물']}

In [ ]:
for keyword in keyword_list:
  test_data[keyword] = -1
test_data

In [10]:
# tokenizing
okt = Okt()
def tokenize(doc):
    # 한글 자음, 모음 제거
    doc = re.sub(pattern='([ㄱ-ㅎㅏ-ㅣ]+)', repl='', string=doc)
    # 특수기호 제거
    doc = re.sub(pattern='[^\w\s]', repl='', string=doc)
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    doc = okt.pos(doc, norm=True, stem=True)
    # 명사, 형용사, 부사, 동사 채택
    token = []
    for i in doc:
        if i[1] == 'Noun' or i[1] == 'Verb' or i[1] == 'Adverb' or i[1] == 'Adjective':
          token.append(i)

    return ['/'.join(t) for t in token]

In [11]:
def vectorize(sentence, encoder):
  l = []
  token = tokenize(sentence)
  l.append(token)
  result = encoder.texts_to_sequences(l)
  return pad_sequences(result, 100)

In [38]:
# 인코더 기존 학습된 모델 사용
import pickle
# with open('drive/My Drive/datasets/encoder_v2.pickle', 'rb') as handle:
#     encoder = pickle.load(handle)
encoder = Tokenizer(num_words=10000)
encoder.fit_on_texts(test_data['review'])
# saving
with open('encoder_v3.pickle', 'wb') as handle:
    pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
def inject_keyword(text, window):
  # 결과값
  result = {'맛':'', '가격': '', '양':'', '품질':'', '간편':'', '외관':'', '웰빙':''}
  sentences = re.split('[,.;?!]+',text) # 문장 분리 by ",.;?"

  # 맛 ~ 웰빙까지
  for key in keyword_list.keys():
    keys = keyword_list[key] # 키워드 + 연관 키워드 리스트
    injected = '' # 추출할 문장

    for k in keys: 
      for sentence in sentences:
        words = sentence.split(' ') # 각 문장 띄어쓰기 분리
        for word in words:
          if k in word:
            index = words.index(word)
            # 키워드에 해당하는 단어의 인덱스를 기준으로 앞 뒤 몇 글자씩 잘라내서 추출
            # window 파라미터로 몇 글자를 잘라낼 것인지 설정
            # 문맥 상 키워드의 이전 단어들 보다 이후에 등장하는 단어들을 더 많이 추출
            for i in range(1,2):
              if index - i >= 0:
                injected += words.pop(index - i)
                injected += ' '
            # for i in range(window+2):
            #   if index + i < len(words):
            #     injected += words.pop(index + i)
            #     injected += ' '
            index = words.index(word)
            for i in range(window):
              if index < len(words):
                injected += words.pop(index) + ' '
            injected += '.'
                 
      result[key] = injected
      
  return result

In [36]:
print(test_data['review'][0])
print(inject_keyword(test_data['review'][0], 5))

포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 상품 포장상태가 안좋았던 건지 아니면 배송하시는 분이 던져서 터진 건지는 모르겠지만  별로네요 그냥 다 버려서 맛은 모르겠어요 맛 만족도 괜찮아요
{'맛': '맛은 모르겠어요 맛 만족도 괜찮아요 .냄새가 나서 자세히 봤더니 사진 .', '가격': '', '양': '', '품질': '포장상태가 안좋았던 건지 아니면 배송하시는 .배송하시는 분이 던져서 터진 건지는 .냄새가 나서 자세히 봤더니 사진 .', '간편': '', '외관': '', '웰빙': ''}


In [43]:
model.predict(vectorize('그냥 다 버렸어요', encoder))

array([[0.93083215]], dtype=float32)

In [ ]:
# 테스트용 3000개 데이터 사용
samples = test_data[:3000]
# 사전 학습된 LSTM 모델 사용
model = load_model('drive/My Drive/datasets/LSTM_binary_v2.h5')
for sample in samples['review']:
  inject_result = inject_keyword(sample, 3)
  for key in keyword_list.keys():
    inject_sentence = inject_result[key]
    # 해당 키워드가 없으면 점수는 -1
    if inject_sentence == '':
      samples.loc[sample, key] = -1
    else:
      samples.loc[sample, key] = model.predict(vectorize(inject_sentence, encoder))[0][0] * 100
samples

In [ ]:
samples[samples['맛'] != -1]

In [ ]:
samples[samples['가격']!= -1]

비고 
 * 아직까지 정확도에 한계가 있다. 키워드 추출 로직도 보완 필요.
 * 키워드 연관 단어 사전 역시 보완 필요
 * POC 나 프로토타입 때 보여주기 용으로는 나쁘지 않은듯.
 * 계산 시간 비용이 많이 든다. 3000개 학습 시 10분 정도, 9만 개 학습 시 5시간

In [45]:
# 키워드 형태소 분석

for key in keyword_list.keys():
  for word in keyword_list[key]:
    print(okt.pos(word))

[('맛', 'Noun')]
[('불맛', 'Noun')]
[('감칠맛', 'Noun')]
[('쓴맛', 'Noun')]
[('향', 'Noun')]
[('단맛', 'Noun')]
[('신맛', 'Noun')]
[('냄새', 'Noun')]
[('끝맛', 'Noun')]
[('짠맛', 'Noun')]
[('꼬순내', 'Noun')]
[('햄맛', 'Noun')]
[('햄향', 'Noun')]
[('닭맛', 'Noun')]
[('불향', 'Noun')]
[('풋내', 'Noun')]
[('조미료', 'Noun')]
[('끝향', 'Noun')]
[('쏘스맛', 'Noun')]
[('닭내', 'Noun')]
[('파맛', 'Noun')]
[('존맛', 'Noun')]
[('핵존맛', 'Noun')]
[('텁텁한맛', 'Noun')]
[('가격', 'Noun')]
[('금액', 'Noun')]
[('값', 'Noun')]
[('단가', 'Noun')]
[('할인', 'Noun')]
[('가성', 'Noun')]
[('득템', 'Noun')]
[('창렬', 'Noun')]
[('혜자', 'Noun')]
[('정가', 'Noun')]
[('사기', 'Noun')]
[('용량', 'Noun')]
[('양은', 'Noun')]
[('양도', 'Noun')]
[('양으로', 'Noun')]
[('개수', 'Noun')]
[('인분', 'Noun')]
[('분량', 'Noun')]
[('물량', 'Noun')]
[('사이즈', 'Noun')]
[('수량', 'Noun')]
[('양', 'Noun'), ('이', 'Josa')]
[('남았', 'Verb')]
[('남아', 'Noun')]
[('많', 'Adjective')]
[('품질', 'Noun')]
[('상태', 'Noun')]
[('아이스', 'Noun')]
[('얼음', 'Noun')]
[('비닐', 'Noun')]
[('아이스팩', 'Noun')]
[('뾱뾱', 'Noun')]
[('뽁뽁이팩', 'Noun')]
[(

In [ ]:
okt.pos(test_data['review'][0])